In [1]:
"""
[V2]
Blend Models:
* tabnet 10 folds 3 seeds using non scored
* 2 stage NN with chris split using nonscored
* deepinsight-efficientnet-v7-b3-infer
* deepinsight-restnest-v2-infer

"""

kernel_mode = False

import os
import numpy as np
import pandas as pd
import time
import random
import math
import glob

from numba import njit
from scipy.optimize import minimize, fsolve

import optuna

import warnings
warnings.filterwarnings('ignore')

import gc
gc.enable()

rand_seed = 1120

optuna_mode = True
study_name = "moa_blend_team_v2"
# n_trials = 500
n_trials = 3000

In [2]:
# !mkdir -p /root/.cache/torch/hub/checkpoints/
# !cp ../input/gen-efficientnet-pretrained/tf_efficientnet_*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v1-resnest50/*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v2-resnest50-output/*.pth /root/.cache/torch/hub/checkpoints/
# !ls -la /root/.cache/torch/hub/checkpoints/

In [3]:
# !cp ../input/kaggle-moa-team/scripts/* .
# !ls -la

In [4]:
dataset_folder = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA/"

# Add your model inference script here
# Tuple Format: (script, oof_filename, output_filename, weight)
model_list = [
    ("moa-inference-data/script_tabnet_ns_oldcv.py",
     "moa-inference-data/oof_tabnet_ns_oldcv.npy",
     "submission_tabnet_ns_oldcv.csv", 0),
    
    ("moa-inference-data/script_nn_ns_newcv.py",
     "moa-inference-data/oof_nn_ns_newcv.npy",
     "submission_nn_ns_newcv.csv", 0),
    
    ("deepinsight_efficientnet_lightning_v7_b3_infer.py",
     "oof_deepinsight_efficientnet_v7_b3_0.014802440208660929.npy",
     "submission_effnet_v7_b3.csv", 0),
    
    ("deepinsight_resnest_lightning_v2_infer.py",
     "oof_deepinsight_ResNeSt_v2_resnest50_0.01455961217985703.npy",
     "submission_resnest_v2.csv", 0),
]

model_path = "." if kernel_mode else dataset_folder

In [5]:
train_features = pd.read_csv(f"{dataset_folder}/train_features.csv",
                             engine='c')
train_labels = pd.read_csv(f'{dataset_folder}/train_targets_scored.csv',
                           engine='c')
train_classes = [c for c in train_labels.columns if c != "sig_id"]

non_control_group_rows = train_features["cp_type"] == "trt_cp"
non_control_group_train_labels = train_labels.loc[
    non_control_group_rows, :].copy().reset_index(drop=True)

submission = pd.read_csv(f'{dataset_folder}/sample_submission.csv')
submission.iloc[:, 1:] = 0

In [6]:
# Reference: https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
# CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
def log_loss_numpy(y_pred, y_true):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, -np.log(y_pred), -np.log(1 - y_pred))
    return loss.mean()


def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes=((0), (0)))
    return log_loss_numpy(oof_blend)


def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.nanmean(-logloss)

In [7]:
total_start = time.time()
if not optuna_mode:
    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Generating submission file from {script} ......")
        infer_start = time.time()
        !python {model_path}/{script}
        infer_elapsed = time.time() - infer_start
        print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

        model_submit = pd.read_csv(output_filename, engine='c')
        print(model_submit.head(5))
        print(model_submit.shape)
        submission.iloc[:, 1:] += weight * model_submit.iloc[:, 1:]
else:
    ## Search Best Blend Weights by Optuna ##
    model_oofs = []

    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")
        
        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        oof_loss = mean_logloss(oof, non_control_group_train_labels[train_classes].values)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    def objective(trial):
        weights = []
        for i in range(len(model_list)):
            weights.append(trial.suggest_float(f"w{i}", 0, 1.0))

        blend = np.zeros(model_oofs[0].shape)
        for i in range(len(model_list)):
            blend += weights[i] * model_oofs[i]
        blend = np.clip(blend, 0, 1.0)

        loss = mean_logloss(blend, non_control_group_train_labels[train_classes].values)
        return loss

    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=0,
        interval_steps=1,
    )
    sampler = optuna.samplers.TPESampler(seed=rand_seed)
    study = optuna.create_study(direction="minimize",
                                pruner=pruner,
                                sampler=sampler,
                                study_name=study_name,
                                storage=f'sqlite:///{study_name}.db',
                                load_if_exists=True)

    study.optimize(objective,
                   n_trials=n_trials,
                   timeout=None,
                   gc_after_trial=True,
                   n_jobs=-1)

    trial = study.best_trial

#     for i, (script, oof_filename, output_filename, _) in enumerate(model_list):
#         optimal_weight = trial.params[f"w{i}"]
#         print(f"Generating submission file from {script} ...... (Weight: {optimal_weight})")
#         infer_start = time.time()
#         !python {model_path}/{script}
#         infer_elapsed = time.time() - infer_start
#         print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

#         model_submit = pd.read_csv(output_filename, engine='c')
#         print(model_submit.head(5))
#         print(model_submit.shape)
#         submission.iloc[:, 1:] += optimal_weight * model_submit.iloc[:, 1:]

    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Loading OOF from moa-inference-data/oof_tabnet_ns_oldcv.npy ......
OOF Validation Loss of moa-inference-data/script_tabnet_ns_oldcv.py: 0.016234

Loading OOF from moa-inference-data/oof_nn_ns_newcv.npy ......
OOF Validation Loss of moa-inference-data/script_nn_ns_newcv.py: 0.016976

Loading OOF from oof_deepinsight_efficientnet_v7_b3_0.014802440208660929.npy ......
OOF Validation Loss of deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

Loading OOF from oof_deepinsight_ResNeSt_v2_resnest50_0.01455961217985703.npy ......


[I 2020-11-26 00:00:04,456] A new study created in RDB with name: moa_blend_team_v2


OOF Validation Loss of deepinsight_resnest_lightning_v2_infer.py: 0.015756



[I 2020-11-26 00:00:05,143] Trial 3 finished with value: 0.020078837351543734 and parameters: {'w0': 0.3244342392654038, 'w1': 0.35729024757605243, 'w2': 0.7315483226774694, 'w3': 0.7311350187571661}. Best is trial 3 with value: 0.016793707536513273.
[I 2020-11-26 00:00:05,373] Trial 0 finished with value: 0.016793707536513273 and parameters: {'w0': 0.17112803265773469, 'w1': 0.5869945148451513, 'w2': 0.5154588557170565, 'w3': 0.057459403016838606}. Best is trial 0 with value: 0.016793707536513273.
[I 2020-11-26 00:00:05,407] Trial 4 finished with value: 0.02257767824027071 and parameters: {'w0': 0.30512581886068535, 'w1': 0.5829215465889119, 'w2': 0.9035973738813379, 'w3': 0.8414269234593335}. Best is trial 0 with value: 0.016793707536513273.
[I 2020-11-26 00:00:05,410] Trial 1 finished with value: 0.017768942681680566 and parameters: {'w0': 0.9599605574897212, 'w1': 0.181871715571699, 'w2': 0.138782181859456, 'w3': 0.4438245137417205}. Best is trial 0 with value: 0.016793707536513273

[I 2020-11-26 00:00:09,445] Trial 31 finished with value: 0.019289431310055064 and parameters: {'w0': 0.6061607035448774, 'w1': 0.31428628123933045, 'w2': 0.13827199670514506, 'w3': 0.9730860741443417}. Best is trial 12 with value: 0.015512901713300322.
[I 2020-11-26 00:00:09,541] Trial 32 finished with value: 0.016448155768681284 and parameters: {'w0': 0.3740009326282652, 'w1': 0.28334227958750197, 'w2': 0.15843863050099316, 'w3': 0.5226647081973624}. Best is trial 12 with value: 0.015512901713300322.
[I 2020-11-26 00:00:09,916] Trial 33 finished with value: 0.01553996744583181 and parameters: {'w0': 0.3992146336095925, 'w1': 0.07877180338737833, 'w2': 0.011318436867767373, 'w3': 0.5493567536727091}. Best is trial 12 with value: 0.015512901713300322.
[I 2020-11-26 00:00:10,152] Trial 34 finished with value: 0.017478244403441673 and parameters: {'w0': 0.41706385182332384, 'w1': 0.0697288765237647, 'w2': 0.5359797878680115, 'w3': 0.5562958674484693}. Best is trial 12 with value: 0.01551

...

[I 2020-11-26 00:08:09,155] Trial 2883 finished with value: 0.015332387054995057 and parameters: {'w0': 0.11986418407878749, 'w1': 0.0006994423251820911, 'w2': 0.34081193341658406, 'w3': 0.5230459274915742}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:09,163] Trial 2884 finished with value: 0.015471132823851406 and parameters: {'w0': 0.22561405772020438, 'w1': 0.0003518763935230207, 'w2': 0.3460789895869675, 'w3': 0.5206914290412307}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:09,525] Trial 2885 finished with value: 0.015430498835581475 and parameters: {'w0': 0.11833083470931281, 'w1': 0.0007620878931035918, 'w2': 0.40916026159551555, 'w3': 0.5245136207366673}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:09,849] Trial 2886 finished with value: 0.01536575941013441 and parameters: {'w0': 0.12321289616284829, 'w1': 0.03402450491808993, 'w2': 0.27984114133397753, 'w3': 0.4909459374683556}. Best is trial

[I 2020-11-26 00:08:15,483] Trial 2916 finished with value: 0.015489635527107233 and parameters: {'w0': 0.2171194267512547, 'w1': 0.017415391761726528, 'w2': 0.38119357233350304, 'w3': 0.47739481995961663}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:15,507] Trial 2914 finished with value: 0.015651360802994826 and parameters: {'w0': 0.21224340532914832, 'w1': 0.00026782048641956575, 'w2': 0.3738713094911189, 'w3': 0.5410276101110569}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:15,508] Trial 2917 finished with value: 0.01538036673779614 and parameters: {'w0': 0.21301845252616267, 'w1': 0.01907749890398034, 'w2': 0.3361364021597474, 'w3': 0.47527097921316075}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:15,514] Trial 2915 finished with value: 0.015776529601416085 and parameters: {'w0': 0.2212385430401292, 'w1': 0.0006980998269532347, 'w2': 0.39068712786475374, 'w3': 0.5448497570092568}. Best is trial 

[I 2020-11-26 00:08:21,413] Trial 2945 finished with value: 0.01548558531938015 and parameters: {'w0': 0.13294710331881643, 'w1': 0.08604822243207035, 'w2': 0.3947176798980696, 'w3': 0.46212705550507593}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:21,791] Trial 2946 finished with value: 0.01535178147143933 and parameters: {'w0': 0.13916119159301038, 'w1': 0.04425924045055852, 'w2': 0.36597229415792365, 'w3': 0.46351610616526856}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:21,892] Trial 2949 finished with value: 0.01553911741273134 and parameters: {'w0': 0.13745960861634632, 'w1': 0.0888038751910874, 'w2': 0.39900494624468125, 'w3': 0.46685168515692865}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:21,905] Trial 2950 finished with value: 0.01535686002188136 and parameters: {'w0': 0.10967147201810529, 'w1': 0.00024512008086740894, 'w2': 0.3582812902748804, 'w3': 0.46201925140100014}. Best is trial 243

[I 2020-11-26 00:08:27,202] Trial 2976 finished with value: 0.015383356641297452 and parameters: {'w0': 0.12237154152298882, 'w1': 0.00047684646587970396, 'w2': 0.2552592461087128, 'w3': 0.520367479998578}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:27,483] Trial 2977 finished with value: 0.015344991302229175 and parameters: {'w0': 0.16868625095628456, 'w1': 0.032464786688807284, 'w2': 0.30759534726535287, 'w3': 0.45350946604484804}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:27,631] Trial 2978 finished with value: 0.015357806422492619 and parameters: {'w0': 0.16802682569933028, 'w1': 0.030440857508528606, 'w2': 0.2860981321174907, 'w3': 0.45221878130056337}. Best is trial 2437 with value: 0.015324438929768825.
[I 2020-11-26 00:08:27,978] Trial 2980 finished with value: 0.015356957134397088 and parameters: {'w0': 0.1676432404635733, 'w1': 0.03533062623712568, 'w2': 0.28612503248170923, 'w3': 0.4522992993199709}. Best is trial 

Number of finished trials: 3000
Best trial:
  Value: 0.015324438929768825
  Params: 
    w0: 0.17220166850775062
    w1: 0.0006140446962010867
    w2: 0.341143317738104
    w3: 0.4853859962663553


In [8]:
total_elapsed = time.time() - total_start
print(f"Total time spent: {total_elapsed/60:.2f} minutes.")

Total time spent: 8.46 minutes.


In [9]:
# Number of finished trials: 500
# Best trial:
#   Value: 0.014158536219669974
#   Params: 
#     w0: 0.3287684605023437
#     w1: 0.2763485706536088
#     w2: 0.3859487453003219

In [10]:
# Number of finished trials: 3000
# Best trial:
#   Value: 0.014287989662394677
#   Params: 
#     w0: 0.4397214034464735
#     w1: 0.5466210401214696

In [11]:
print(submission.shape)
submission

(3982, 207)


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001140,0.001169,0.001982,0.016637,0.028251,0.004118,0.003356,0.003270,0.000252,...,0.001502,0.000647,0.002621,0.001826,0.000815,0.000746,0.000761,0.002113,0.002082,0.001310
1,id_001897cda,0.000194,0.000270,0.000732,0.000334,0.000743,0.001452,0.002962,0.006127,0.058353,...,0.000328,0.000143,0.001729,0.000768,0.004997,0.000174,0.004149,0.000435,0.000187,0.002191
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000512,0.000411,0.001805,0.019714,0.016225,0.004170,0.003897,0.003916,0.000429,...,0.000819,0.001271,0.001750,0.011008,0.031156,0.000446,0.004321,0.002950,0.002086,0.003045
4,id_0027f1083,0.003641,0.001858,0.001494,0.015493,0.023506,0.004922,0.005166,0.001399,0.000332,...,0.001317,0.000739,0.002620,0.001143,0.001230,0.000870,0.001134,0.001481,0.000477,0.001661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.000262,0.000490,0.000595,0.002090,0.004227,0.000717,0.000626,0.001199,0.000385,...,0.000660,0.010027,0.002168,0.349161,0.007396,0.001034,0.004639,0.000505,0.000806,0.000355
3978,id_ff925dd0d,0.004736,0.003249,0.001048,0.010602,0.020970,0.006351,0.005969,0.002876,0.000273,...,0.000628,0.000635,0.003043,0.001603,0.001152,0.001074,0.002228,0.001827,0.000363,0.000892
3979,id_ffb710450,0.004125,0.002012,0.000942,0.009287,0.030860,0.009721,0.003308,0.001775,0.000248,...,0.000615,0.000696,0.002354,0.000970,0.001489,0.000763,0.000839,0.001155,0.000323,0.001657
3980,id_ffbb869f2,0.001578,0.000898,0.001135,0.027620,0.025039,0.003751,0.009288,0.002486,0.000657,...,0.000972,0.000554,0.002377,0.001237,0.001722,0.000495,0.000787,0.002356,0.000556,0.002994


In [12]:
submission.to_csv('submission.csv', index=False)

In [13]:
!rm ./*.py
!ls -la

total 65024
drwxr-xr-x 4 root root     4096 Nov 22 16:31 .
drwxr-xr-x 6 root root     4096 Nov 22 16:07 ..
---------- 1 root root   234779 Nov 22 16:31 __notebook__.ipynb
drwxr-xr-x 2 root root     4096 Nov 22 16:08 checkpoints
-rw-r--r-- 1 root root 16575670 Nov 22 16:31 submission.csv
-rw-r--r-- 1 root root 16577393 Nov 22 16:16 submission_effnet_v7_b3.csv
-rw-r--r-- 1 root root 16569497 Nov 22 16:23 submission_resnest_v1.csv
-rw-r--r-- 1 root root 16598162 Nov 22 16:30 submission_resnest_v2.csv
drwxr-xr-x 2 root root     4096 Nov 22 16:08 wandb
